在自然语言处理领域中，分词和提取关键词都是对文本处理时通常要进行的步骤。用Python语言对英文文本进行预处理时可选择NLTK库，中文文本预处理可选择jieba库。jieba的详细使用请看[文档](https://github.com/fxsjy/jieba)

## 1. 采用的算法：

1.基于`前缀词典`实现高效的词图扫描，生成句子中汉字所有可能成词情况所构成的`有向无环图(DAG)`

2.采用了`动态规划`查找`最大概率路径`, 找出基于词频的最大切分组合

3.对于未登录词，采用了基于汉字成词能力的`HMM`模型，使用了`Viterbi`算法

## 2. 基本分词函数和用法

**jieba分词支持三种模式：**

* `精确模式`：适合将句子最精确的分开，适合文本分析；这是默认模式.

* `全模式`：把句子中所有可以成词的词语都扫描出来，速度快，但是不能解决歧义；

* `搜索引擎模式`：在精确模式的基础上，对长词再次进行切分，提高召回率，适用于搜索引擎分词；

**jieba 提供以下两种基本方法来进行分词：**

`jieba.cut` 方法接受三个输入参数:

* 需要分词的字符串
* `cut_all` 参数用来控制是否采用全模式. If True, 采用全模式. Otherwise, 采用精确模式.
* `HMM` 参数用来控制是否使用 HMM 模型

`jieba.cut_for_search` 运用搜索引擎模式进行分词. 该方法接受两个参数：

* 需要分词的字符串
* `HMM` 参数用来控制是否使用 HMM 模型。

`jieba.cut` 和 `jieba.cut_for_search` 方法返回的结构都是一个可迭代的 generator，可以使用 for 循环来获得分词后得到的每一个词语

如果想直接使用list, 可以使用 `jieb.lcut` 以及 `jieba.lcut_for_search` 方法。 这两个方法都是直接返回list

还有另外一个方法 `jieba.Tokenizer(dictionary=DEFUALT_DICT)` 用于新建自定义分词器，可用于同时使用不同字典，
jieba.dt 为默认分词器，所有全局分词相关函数都是该分词器的映射。

In [1]:
import jieba

In [24]:
# 运用精确模式进行分词 via cut_all=False
wordlist = jieba.cut('我爱学习自然语言处理', cut_all=False)
print('(默认)精确模式: ' + '/ '.join(wordlist))

(默认)精确模式: 我/ 爱/ 学习/ 自然语言/ 处理


In [25]:
# 运用全模式进行分词 via cut_all=True
wordlist = jieba.cut('我爱学习自然语言处理', cut_all=True)
print('全模式: ' + '/ '.join(wordlist))

全模式: 我/ 爱/ 学习/ 自然/ 自然语言/ 语言/ 处理


In [26]:
# 运用搜索引擎模式进行分词 via jieba.cut_for_search
wordlist = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在哈佛大学深造")  
print('搜索引擎模式: ' + '/ '.join(wordlist))

搜索引擎模式: 小明/ 硕士/ 毕业/ 于/ 中国/ 科学/ 学院/ 科学院/ 中国科学院/ 计算/ 计算所/ ，/ 后/ 在/ 哈佛/ 大学/ 哈佛大学/ 深造


## 3. 添加用户自定义字典：

很多时候我们需要针对自己的场景进行分词，会有一些特定领域的专有词汇，以便包含词库中没有的词。虽然jieba分词有新词识别能力，但是自行添加新词可以保证更高的正确率。

1. 用 `jieba.load_userdict(file_name)` 加载用户自定义词典。 在使用的时候，词典的格式和jieba分词器本身的分词器中的词典格式必须保持一致：
    * 一个词占一行，每一行分成三部分，一部分为`词语`，一部分为`词频`，最后为`词性`（可以省略），用空格隔开。


2. 少量的词汇可以自己用下面方法手动添加：
    * 用 `add_word(word, freq=None, tag=None)` 和 `del_word(word)` 在程序中动态修改词典
    * 用 `suggest_freq(segment, tune=True)` 可调节单个词语的词频，使其能（或不能）被分出来。

In [31]:
comment = '扫地后基本是找不到充电座的，或者电量不足的时候自动回充也基本上找不到充电座。扫地存在漏扫，优点是声音小'

In [32]:
wordlist = jieba.cut(comment, cut_all=False)
print('(默认)精确模式: \n' + '/ '.join(wordlist))

(默认)精确模式: 
扫地/ 后/ 基本/ 是/ 找/ 不到/ 充电/ 座/ 的/ ，/ 或者/ 电量/ 不足/ 的/ 时候/ 自动/ 回充/ 也/ 基本上/ 找/ 不到/ 充电/ 座/ 。/ 扫地/ 存在/ 漏扫/ ，/ 优点/ 是/ 声音/ 小


In [37]:
file = open('userdict.txt')
user_dict = file.read()
print(user_dict)

充电座 1
回充 1



In [38]:
jieba.load_userdict('userdict.txt')

In [39]:
wordlist = jieba.cut(comment, cut_all=False)
print('(默认)精确模式: \n' + '/ '.join(wordlist))

(默认)精确模式: 
扫地/ 后/ 基本/ 是/ 找/ 不到/ 充电座/ 的/ ，/ 或者/ 电量/ 不足/ 的/ 时候/ 自动/ 回充/ 也/ 基本上/ 找/ 不到/ 充电座/ 。/ 扫地/ 存在/ 漏扫/ ，/ 优点/ 是/ 声音/ 小


## 4. 关键词抽取

关键词提取是文本分析的关键一步. 只有关键词抽取并且进行词向量化之后，才好进行下一步的文本分析，可以说这一步是自然语言处理技术中文本处理最基础的一步。

关键词抽取算法在`jieba.analyse`模块中，主要有两种算法：

1.基于 TF-IDF 算法的关键词抽取(import jieba.analyse). 可以参考[详解自然语言处理之TF-IDF模型和python实现](http://mp.weixin.qq.com/s?__biz=MzIwNzYzNjA1OQ==&mid=2247484212&idx=1&sn=8a1f402fcdbf5c982c71859ce7e08c25&chksm=970e1000a079991611e03948ba0a74e6e45f6c2d64dcc0d65d9848d266ef751a0cb6a48864d2&scene=21#wechat_redirect)

`jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())`
* `sentence` 为待提取的文本；
* `topK` 为返回几个 TF/IDF 权重最大的关键词，默认值为 20；
* `withWeight` 为是否一并返回关键词权重值，默认值为 False；
* `allowPOS` 仅包括指定词性的词，默认值为空，即不筛选。

关键词提取所使用逆向文件频率（IDF）文本语料库可以切换成自定义语料库的路径：jieba.analyse.set_idf_path(file_name) # file_name为自定义语料库的路径

关键词提取所使用停止词（Stop Words）文本语料库可以切换成自定义语料库的路径：jieba.analyse.set_stop_words(file_name) # file_name为自定义语料库的路径

In [48]:
import jieba.analyse as analyse

print(comment)
for word, weight in analyse.extract_tags(comment, topK=10, withWeight=True, allowPOS=()):
    print(word, weight)
# print("  ".join(analyse.extract_tags(comment, topK=10, withWeight=False, allowPOS=())))

扫地后基本是找不到充电座的，或者电量不足的时候自动回充也基本上找不到充电座。扫地存在漏扫，优点是声音小
充电座 1.3283075003222222
扫地 1.0350789081455556
回充 0.6641537501611111
漏扫 0.6641537501611111
不到 0.59291325201
电量 0.520448786601111
优点 0.413116074905
自动 0.3600720593922222
基本上 0.35528807651333333
不足 0.31737901157111115


2.基于TextRank算法的关键词抽取

`jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=(‘ns’, ‘n’, ‘vn’, ‘v’))` 

* 提取关键词，接口相同，注意默认过滤词性。
* 算法基本思路：
    * 将待抽取关键词的文本进行分词；
    * 以固定窗口大小(默认为5，通过span属性调整)，词之间的共现关系，构建图；
    * 计算图中节点的PageRank，注意是无向带权图

In [50]:
for word, weight in analyse.textrank(comment, topK=10, withWeight=True, allowPOS=('ns', 'n', 'vn','v')):
    print(word, weight)
# print("  ".join(analyse.textrank(comment, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn','v'))))

扫地 1.0
不到 0.8302322174672403
漏扫 0.811010851047124
优点 0.8015471019947196
存在 0.7926304152728431
自动 0.7448169853914659
时候 0.7404494216857908
基本上 0.6990560686302163
电量 0.5156251901480693
基本 0.43221658571106614


## 5. 词性标注

* `jieba.posseg.POSTokenizer(tokenizer=None)` 新建自定义分词器，tokenizer 参数可指定内部使用的 jieba.Tokenizer 分词器。jieba.posseg.dt 为默认词性标注分词器。
* 标注句子分词后每个词的词性，采用和 ictclas 兼容的标记法。


In [55]:
import jieba.posseg as pseg

comment = '扫地后基本是找不到充电座的，或者电量不足的时候自动回充也基本上找不到充电座。扫地存在漏扫，优点是声音小'
words = pseg.cut(comment)
for word, flag in words:
    print('%s %s' % (word, flag))

扫地 n
后 f
基本 n
是 v
找 v
不到 v
充电座 x
的 uj
， x
或者 c
电量 n
不足 a
的 uj
时候 n
自动 vn
回充 x
也 d
基本上 n
找 v
不到 v
充电座 x
。 x
扫地 n
存在 v
漏扫 v
， x
优点 n
是 v
声音 n
小 a
